In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import os
import seaborn as sns
import random 
import statsmodels.api as sm

import scipy.stats as stats
import cartopy.crs as ccrs
import pylab
import tables

sns.set_theme(style="white")
sns.set_style("white")
random.seed(10)

## Load single member

In [ ]:
tracks1 = pd.read_hdf('/gf3/predict2/AWH012_LEACH_NASTORM/DATA/postproc/tracks/MED-R/ENS/EU025/sfc/cf/1_2022-02-10.h5')
# tracks1.head()
# tracks1.date.values
tracks1.columns

In [ ]:
pd.unique(tracks1.uid)

The following track could be the Eunice track in this member, uid = '202202022', also uid = '202202007'. It is longer than one day, gets sufficiently strong and is created in the area that we would expect.

In [ ]:
tracks1[tracks1.uid=='202202007'][['uid', 'date', 'lat', 'long', 'p_cent']] 

In [ ]:
print('Minimum pressure = ', min(tracks1[tracks1.uid=='202202022'].p_cent))
print('Genesis latitude = ', tracks1[tracks1.uid=='202202022'].lat.values[0])
print('Genesis longitude = ', tracks1[tracks1.uid=='202202022'].long.values[0])

In [ ]:
fig,ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))

sns.lineplot(data=tracks1,x='long',y='lat',hue='uid',legend=False)
sns.lineplot(data=tracks1[tracks1.uid == '202202007'],x='long',y='lat',color='red',legend=False)
sns.lineplot(data=tracks1[tracks1.uid == '202202022'],x='long',y='lat',color='red',legend=False)
ax.coastlines()

In [ ]:
early_date = pd.Timestamp(2022,2,16, 0)
late_date =  pd.Timestamp(2022,2,18, 18)

# Filter uid
eunice_ids = [] # fill with those ids that could be Eunice
for id_ in pd.unique(tracks1.uid): 
    # print(id_)
    data_for_id = tracks1[tracks1.uid == id_]
    # print('Genesis data: ', data_for_id.date[0], data_for_id.long[0], data_for_id.lat[0])
    min_pres = min(data_for_id.p_cent) < 99000  # in Pa not hPa 
    gen_lat = (data_for_id.lat[0] > 40) & (data_for_id.lat[0] < 65)
    gen_long = (data_for_id.long [0] > -45) & (data_for_id.long[0] < 0)
    gen_time = ((data_for_id.date[0] >=  early_date) & (data_for_id.date[0] < late_date))

    # print(min_pres, gen_lat, gen_long, gen_time)
    
    if min_pres & gen_lat & gen_long & gen_time:
        eunice_ids.append(id_)

In [ ]:
eunice_ids

## Filter tracks in all members

In [ ]:
## Load all members, ensemble i.e current climate
dates = ['10', '14', '16']
cfpf = ['cf', 'pf']

# Initialise empty data frame
tracks_curr = pd.DataFrame({'uid' : [],
                              'date' : [], 
                              'lat' : [], 
                              'long' : [], 
                              'p_cent' : []})
members = np.arange(1,51,1)

for date in dates:
    for c in cfpf:
        if c == 'pf': 
            for member in members:
                tracks = pd.read_hdf('/gf3/predict2/AWH012_LEACH_NASTORM/DATA/postproc/tracks/MED-R/ENS/EU025/sfc/'+c+'/1_2022-02-'+date+'_mem'+str(member)+'.h5')
                tracks.uid = tracks.uid.values+'_init'+str(date)+'_mem'+str(member)
                tracks_curr = pd.merge(tracks_curr, 
                                       tracks[['uid', 'date', 'lat', 'long', 'p_cent']], 
                                       how = 'outer')
        else: 
            tracks = pd.read_hdf('/gf3/predict2/AWH012_LEACH_NASTORM/DATA/postproc/tracks/MED-R/ENS/EU025/sfc/'+c+'/1_2022-02-'+date+'.h5')
            tracks.uid = tracks.uid.values+'_init'+str(date)+'_memCF'
            tracks_curr = pd.merge(tracks_curr, 
                                   tracks[['uid', 'date', 'lat', 'long', 'p_cent']], 
                                   how = 'outer')

In [ ]:
early_date = pd.Timestamp(2022,2,16, 0)
late_date =  pd.Timestamp(2022,2,18, 18)

# Filter uid
eunice_ids_curr = [] # fill with those ids that could be Eunice
for id_ in pd.unique(tracks_curr.uid): 
    data_for_id = tracks_curr[tracks_curr.uid == id_]
    min_pres = min(data_for_id.p_cent) < 99000  # in Pa not hPa 
    gen_lat = (data_for_id.lat.values[0] > 40) & (data_for_id.lat.values[0] < 65)
    gen_long = (data_for_id.long.values[0] > -45) & (data_for_id.long.values[0] < 0)
    gen_time = ((data_for_id.date.values[0] >=  early_date) & (data_for_id.date.values[0] < late_date))

    
    if min_pres & gen_lat & gen_long & gen_time:
        eunice_ids_curr.append(id_)

In [ ]:
## Load all members for experiments

experiments = ['pi', 'incr']
cfpf = ['cf', 'pf']

# Initialise empty data frame
tracks_pi = pd.DataFrame({'uid' : [],
                          'date' : [], 
                          'lat' : [], 
                          'long' : [], 
                          'p_cent' : []})

tracks_incr = tracks_pi.copy()

dates = {'pi' : ['b2nn_2022-02-14', 'b2ns_2022-02-16', 'b2nq_2022-02-10'], 
               'incr' : ['b2no_2022-02-14', 'b2nr_2022-02-10', 'b2nt_2022-02-16']}  # members for incr and pi runs
members = np.arange(1,51,1)
for exp in experiments:
    for date in dates[exp]:
        for c in cfpf:
            if c == 'pf': 
                for member in members:
                    tracks = pd.read_hdf('/gf3/predict2/AWH012_LEACH_NASTORM/DATA/postproc/tracks/MED-R/EXP/'+exp+'/EU025/sfc/'+c+'/'+date+'_mem'+str(member)+'.h5')
                    tracks.uid = tracks.uid.values+'_init'+str(date)+'_mem'+str(member)
                    if exp == 'pi':
                        tracks_pi = pd.merge(tracks_pi, 
                                            tracks[['uid', 'date', 'lat', 'long', 'p_cent']], 
                                            how = 'outer')
                    elif exp == 'incr':
                        tracks_incr = pd.merge(tracks_incr, 
                                            tracks[['uid', 'date', 'lat', 'long', 'p_cent']], 
                                            how = 'outer')
            else: 
                tracks = pd.read_hdf('/gf3/predict2/AWH012_LEACH_NASTORM/DATA/postproc/tracks/MED-R/EXP/'+exp+'/EU025/sfc/'+c+'/'+date+'.h5')
                tracks.uid = tracks.uid.values+'_init'+str(date)+'_memCF'
                if exp == 'pi':
                    tracks_pi = pd.merge(tracks_pi, 
                                        tracks[['uid', 'date', 'lat', 'long', 'p_cent']], 
                                        how = 'outer')
                elif exp == 'incr':
                    tracks_pi = pd.merge(tracks_incr, 
                                        tracks[['uid', 'date', 'lat', 'long', 'p_cent']], 
                                        how = 'outer')